In [1]:
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
n_inds = 30
scale = "07"
fps = 30
seq_length = 30
full_length = 600

folders = ['0084', '0115', '0120']

angle_thresh = 15
time_thresh = 5
thresh_folder = 'angle' + str(angle_thresh) + '_time' + str(time_thresh)

In [3]:
for i,name in enumerate(folders):
    tmp = pd.read_csv('/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/DirectionalCorrelation/Data/Output/golden_shiners/' + str(n_inds) + '_fish/' + name + '/' + thresh_folder + '/pairwise_' + scale + '.csv')
    tmp2 = pd.read_csv('/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/DirectionalCorrelation/Data/Output/golden_shiners/' + str(n_inds) + '_fish/' + name + '/' + thresh_folder + '/individual_' + scale + '.csv')
    tmp['vid_id'] = i
    tmp2['vid_id'] = i
    
    pair_drops = ['Unnamed: 0', 'ang_area', 'tau']
    ind_drops = ['Unnamed: 0', 'size', 'dev_gspeed', 'dev_gacc', 'rx', 'ry', 'lscore', 'lfinal']
    tmp.drop(pair_drops, axis=1, inplace=True)
    tmp2.drop(ind_drops, axis=1, inplace=True)
    
    if i != 0:
        pair1 = np.concatenate((pair1,tmp), axis=0)
        ind1 = np.concatenate((ind1,tmp2), axis=0)
    else:
        pair1 = tmp
        ind1 = tmp2
        cols = list(tmp)
        col_ind = list(tmp2)
        
pair1 = pd.DataFrame(pair1)
pair1.columns = cols

ind1 = pd.DataFrame(ind1)
ind1.columns = col_ind

In [4]:
n_inds = 10
folders = ['0066', '0105', '0126']

In [5]:
for i,name in enumerate(folders):
    tmp = pd.read_csv('/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/DirectionalCorrelation/Data/Output/golden_shiners/' + str(n_inds) + '_fish/' + name + '/' + thresh_folder + '/pairwise_' + scale + '.csv')
    tmp2 = pd.read_csv('/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/DirectionalCorrelation/Data/Output/golden_shiners/' + str(n_inds) + '_fish/' + name + '/' + thresh_folder + '/individual_' + scale + '.csv')
    tmp['vid_id'] = i+3
    tmp2['vid_id'] = i+3
    
    pair_drops = ['Unnamed: 0', 'ang_area', 'tau']
    ind_drops = ['Unnamed: 0', 'size', 'dev_gspeed', 'dev_gacc', 'rx', 'ry', 'lscore', 'lfinal']
    tmp.drop(pair_drops, axis=1, inplace=True)
    tmp2.drop(ind_drops, axis=1, inplace=True)
    
    if i != 0:
        pair2 = np.concatenate((pair2,tmp), axis=0)
        ind2 = np.concatenate((ind2,tmp2), axis=0)
    else:
        pair2 = tmp
        ind2 = tmp2
        cols = list(tmp)
        col_ind = list(tmp2)
        
pair2 = pd.DataFrame(pair2)
pair2.columns = cols

ind2 = pd.DataFrame(ind2)
ind2.columns = col_ind

In [6]:
ind1 = ind1.rename(index=str, columns={'speed':'f_speed', 'acceleration':'f_acc'})
ind2 = ind2.rename(index=str, columns={'speed':'f_speed', 'acceleration':'f_acc'})

rev_ind1 = ind1.rename(index=str, columns={'f_id':'n_id', 'f_speed':'n_speed', 'f_acc':'n_acc'})
rev_ind2 = ind2.rename(index=str, columns={'f_id':'n_id', 'f_speed':'n_speed', 'f_acc':'n_acc'})

In [7]:
pairs = pd.concat([pair1,pair2], axis=0, sort=True)
inds = pd.concat([ind1,ind2], axis=0, sort=True)
rev_inds = pd.concat([rev_ind1, rev_ind2], axis=0, sort=True)

In [8]:
del tmp
del tmp2

del pair1
del pair2
del ind1
del ind2
del rev_ind1
del rev_ind2

gc.collect()

16

In [9]:
df = pd.merge(pairs,inds)
df = pd.merge(df,rev_inds)

del pairs
del inds
del rev_inds

gc.collect()

0

### Isolate timeseries segments where 
#### a pair switches from having a leader-follower relationship to not having one—segments_upward (0 -> 1)
#### OR
#### a pair switches from not having a leader-follower relationship to having one—segments_downward (1 -> 0)

In [10]:
segments_upward = pd.DataFrame()
e = 0

for i in np.unique(df['vid_id']):
    for j in np.unique(df['f_id']):
        for k in np.unique(df['n_id']):
            tmp = df[(df['vid_id'] == i) & (df['f_id'] == j) & (df['n_id'] == k) & (df['cc'] == 1)].reset_index()
            tmp = tmp[(tmp['frame'] - tmp['frame'].shift() != 1) & (tmp['frame'].shift(-seq_length) - tmp['frame'] == seq_length)].reset_index()
            if tmp.empty == True:
                break
            else:
                for l in np.unique(tmp['frame']):
                    if l > int(scale)*fps//2+full_length and l < np.max(tmp['frame']) - (int(scale)*fps//2+full_length):
                        tmp2 = df[(df['vid_id'] == i) & (df['f_id'] == j) & (df['n_id'] == k) & (df['frame'] >= l-seq_length) & (df['frame'] < l+seq_length)].reset_index()
                        if len(tmp2[tmp2['cc'] == 0]) == seq_length:
                            tmp2 = df[(df['vid_id'] == i) & (df['f_id'] == j) & (df['n_id'] == k) & (df['frame'] >= l-full_length) & (df['frame'] < l+full_length)].reset_index()
                            tmp2 = tmp2.drop('index', axis=1)
                            tmp2['frame'] = np.arange(-full_length,full_length)
                            tmp2['event'] = e
                            tmp2 = tmp2.drop(['vid_id'], axis=1)
                            
                            segments_upward = pd.concat([segments_upward,tmp2])
                            e+=1
    print(i)
                            
segments_upward.to_csv('/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/DirectionalCorrelation/Data/Output/golden_shiners/' + thresh_folder + '/segup_' + scale + '_seq' + str(int(seq_length/fps)) + '.csv', mode='w')

0.0
1.0
2.0
3.0
4.0
5.0


OSError: Cannot save file into a non-existent directory: '/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/DirectionalCorrelation/Data/Output/golden_shiners/angle15_time5'

In [10]:
segments_downward = pd.DataFrame()
e = 0

for i in np.unique(df['vid_id']):
    for j in np.unique(df['f_id']):
        for k in np.unique(df['n_id']):
            tmp = df[(df['vid_id'] == i) & (df['f_id'] == j) & (df['n_id'] == k) & (df['cc'] == 1)].reset_index()
            tmp = tmp[(tmp['frame'].shift(-1) - tmp['frame'] != 1) & (tmp['frame'] - tmp['frame'].shift(seq_length) == seq_length)].reset_index()
            if tmp.empty == True:
                break
            else:
                for l in np.unique(tmp['frame']):
                    if l > int(scale)*fps//2+full_length and l < np.max(tmp['frame']) - (int(scale)*fps//2+full_length):
                        tmp2 = df[(df['vid_id'] == i) & (df['f_id'] == j) & (df['n_id'] == k) & (df['frame'] > l-seq_length) & (df['frame'] <= l+seq_length)].reset_index()
                        if len(tmp2[tmp2['cc'] == 0]) == seq_length:
                            tmp2 = df[(df['vid_id'] == i) & (df['f_id'] == j) & (df['n_id'] == k) & (df['frame'] > l-full_length) & (df['frame'] <= l+full_length)].reset_index()
                            tmp2 = tmp2.drop('index', axis=1)
                            tmp2['frame'] = np.arange(-full_length,full_length)
                            tmp2['event'] = e
                            
                            tmp2 = tmp2.drop(['vid_id'], axis=1)
                            
                            segments_downward = pd.concat([segments_downward,tmp2])
                            e+=1
    print(i)
                            
segments_downward.to_csv('/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/DirectionalCorrelation/Data/Output/golden_shiners/' + thresh_folder + '/segdown_' + scale + '_seq' + str(int(seq_length/fps)) + '.csv', mode='w')

0.0
1.0
2.0
3.0
4.0
5.0
